# 1. 环境配置

## 1.1 python 环境准备

In [ ]:
! pip install openai==2.11.0 dashscope==1.25.4 langchain-classic==1.0.0 langchain==1.1.3 langchain-community==0.4.1 langchain-openai==1.1.3 arxiv==2.3.1

## 1.2 大模型密钥准备

请根据第一章内容获取相关平台的 API KEY，如若未在系统变量中填入，请将 API_KEY 信息写入以下代码（若已设置请忽略）：

In [ ]:
import os

# os.environ["OPENAI_API_KEY"] = "sk-xxxxxxxx"
# os.environ["DASHSCOPE_API_KEY"] = "sk-yyyyyyyy"

## 1.3 实践代码

为了能够顺利的演示内置中间件的使用详情，这里我们使用一段简单的智能体代码演示：

In [1]:
from langchain_community.chat_models import ChatTongyi
import os
llm = ChatTongyi(api_key=os.environ.get("DASHSCOPE_API_KEY"), model="qwen-turbo")

from langchain_community.agent_toolkits.load_tools import load_tools
tools = load_tools(["arxiv"])

from langgraph.checkpoint.memory import InMemorySaver 
memory = InMemorySaver()

from langchain.agents import create_agent
agent = create_agent(model=llm, 
                     tools=tools, 
                     system_prompt="You are a helpful assistant", 
                     checkpointer=memory)

result1 = agent.invoke({"messages": [{"role": "user", "content": "请使用 arxiv 工具查询论文编号 1605.08386"}]}, config={"configurable": {"thread_id": "user_1"}})
print(result1["messages"][-1].content)

论文编号 1605.08386 的信息如下：

- **发表日期**：2016-05-26
- **标题**：Heat-bath random walks with Markov bases
- **作者**：Caprice Stanley, Tobias Windisch
- **摘要**：研究了格点上的图，其边来自于有限集的允许移动。我们证明了在固定整数矩阵的纤维上，这些图的直径可以被常数上界限制。然后研究了这些图上的热浴随机游走的混合行为。还给出了移动集的显式条件，使得热浴随机游走（Glauber动力学的一种推广）在固定维度下是一个扩展器。


# 2. 多智能体应用开发

## 2.1 简介

在前面的学习中，我们主要介绍了：
- LangChain 的内置工具
- 自定义工具的实现方式
- MCP 工具的实现方式

此外，在之前关于内置中间件的介绍中，也有许多与“工具优化”密切相关的内容，例如：
- ToolCallLimitMiddleware：限制工具调用频率
- ToolRetryMiddleware：工具失败时自动重试
- LLMToolEmulator：用LLM模拟工具执行

其中，最后一个 LLMToolEmulator 体现出一个关键思路：大模型本身，就可以是一种能力极强的“工具”。除了输出文本字符串以外，通过结构化输出可辅助我们完成更多复杂任务。

很多人可能会疑惑：既然 agent 的“大脑”本身就是一个大模型，为什么还需要再将另一个大模型作为工具来使用呢？主要有以下几个原因：
- 首先，主智能体模型通常是固定的，难以根据不同任务动态调整。而如果我们将大模型作为一个独立工具，就可以为它自定义专属的系统提示词，甚至使用个性化微调的模型，从而让其更贴合具体任务的需求。
- 其次，所谓的“大模型工具”其实也可以是一个带有工具链的智能体。这样，我们就能将主智能体的一部分功能拆分出去，让各个子模型各司其职，从而形成更高效的分工协作体系。
- 最后，agent 在执行多轮推理时会积累大量上下文信息。这些上下文在某些场景下可能会干扰模型的输出，导致结果不够聚焦。将大模型或智能体作为工具调用，可以有效隔离上下文，使它只聚焦于当前任务，从而提高输出的准确性和可控性。

## 2.2 大模型工具
假如单纯希望 LLM 作为工具，可以使用直接定义函数工具实现：

In [9]:
from langchain_core.tools import tool

@tool
def final_tool(request: str) -> str:
    '''请在所有流程结束后，使用该工具返回最终结果，无论前面工具如何，最后一定要使用该工具'''
    prompt = f"请用100字总结该部分内容，并返回最终结果，内容为：{request}"
    result = llm.invoke(prompt)
    return result.content


tools = [final_tool]

agent = create_agent(model=ChatTongyi(model="qwen-max"), 
           tools=tools, 
           system_prompt="请合理的使用工具完成任务")

result1 = agent.invoke({"messages": [{"role": "user", "content": "请总结一下什么是 AI，最后务必使用工具"}]})
print(result1)
print(result1["messages"][-1].content)

{'messages': [HumanMessage(content='请总结一下什么是 AI，最后务必使用工具', additional_kwargs={}, response_metadata={}, id='d8e4d64b-2aff-4354-beed-c67bb0e98a5d'), AIMessage(content='人工智能（Artificial Intelligence，简称 AI）是指由人制造出来的具有一定智能的系统或软件，能够理解复杂的信息、学习新的知识、进行推理和解决问题，以及通过感知环境来采取行动。AI 的目标是使机器能够执行通常需要人类智慧才能完成的任务，比如视觉识别、语言理解、决策制定等。\n\n在技术层面，AI 涵盖了多个领域，包括但不限于机器学习、深度学习、自然语言处理、计算机视觉、语音识别和机器人学。这些技术使得机器可以从大量的数据中自动学习模式，并利用学到的知识来做出预测或者决定，而无需人为地对每个可能的情况编程。\n\n随着计算能力的增长和大数据的可用性增强，AI 正变得越来越强大，并且在许多行业中得到应用，如医疗保健、金融服务、交通运输、娱乐、教育等等。它正在改变我们生活和工作的方式，为解决复杂问题提供了新的可能性。', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"request": "总结了什么是AI，包括其定义、涵盖的技术领域以及对社会的影响。"}', 'name': 'final_tool'}, 'id': 'call_d54a1d6d74f240edbe80cf', 'index': 0, 'type': 'function'}]}, response_metadata={'model_name': 'qwen-max', 'finish_reason': 'tool_calls', 'request_id': '18a7f8eb-1aab-4bb4-b960-488c908999e4', 'token_usage': {'input_tokens': 262, 'output_tokens': 223, 'prompt_tokens_details': {'cached_tokens': 0}, 'total_tokens': 485

对于一个智能体而言，假如我们提供给其各种各样的工具，包括处理日历、邮件、搜索、翻译等功能。但是当我们运行的轮数变多时，可能会出现以下问题：
- 认知负担过重
- 上下文信息太杂
- 缺乏专业能力
- 控制流程太复杂

所以其实我们可以对一个“大而全”的智能体进行拆分，用多个小而专的 Agent 协作来完成一个大任务，而不是让一个万能 Agent 全部搞定。

## 2.3 Tool Calling 多智能体系统
假如使用 Tool Calling 的方式构建多智能体系统，通常分为以下几步：
- 创建底层工具
- 构建子智能体（Sub-agent）
- 将子代理包装成工具（给 Supervisor 使用）
- 构建 Supervisor Agent（调度者）
- 调用 Supervisor Agent 

### 2.3.1 创建底层工具

其实前面很多节课程里我们都已经提到了什么是工具。在 Agent 中，Tool 是可以被 agent 调用的“动作”——通常对应一个函数。

比如对于前面提到的任务：“请帮我安排明天的行程，并发邮件告诉我朋友。”

我们可以将这个任务的实现拆分为以下几个工具：
- 查询空闲时间 get_available_time_slots
- 创建日程 create_calendar_event
- 发送邮件 send_email

当然由于这里只是演示，后续的工具都只是模拟真实场景而已，但是大家可以在接下来的工具中添加真实的逻辑。

#### 查询空闲时间 get_available_time_slots ：

In [10]:
@tool
def get_available_time_slots(
  attendees: list[str],
  date: str, # ISO format: "2024-01-15"
  duration_minutes: int
) -> list[str]:
  """Check calendar availability for given attendees on a specific date."""
  return ["09:00", "14:00", "16:00"]

#### 创建日程 create_calendar_event ：

In [11]:
@tool
def create_calendar_event(
  title: str,
  start_time: str, # ISO format: "2024-01-15T14:00:00"
  end_time: str,  # ISO format: "2024-01-15T15:00:00"
  attendees: list[str], # email addresses
  location: str = ""
) -> str:
  """Create a calendar event. Requires exact ISO datetime format."""
  return f"Event created: {title} from {start_time} to {end_time} with {len(attendees)} attendees"

#### 发送邮件 send_email ：

In [12]:
@tool
def send_email(
  to: list[str],   # email addresses
  subject: str,
  body: str,
  cc: list[str] = []
) -> str:
  """Send an email via email API. Requires properly formatted addresses."""
  return f"Email sent to {', '.join(to)} - Subject: {subject}"

### 2.3.2 构建子智能体（sub-agents）
所谓的子智能体其实是一个只负责「一个垂直任务」的智能体，它自己可以使用工具，但不与用户直接对话。

那我们这里可以根据任务需要将其拆分为两个子智能体：
- 日程管理子智能体 calendar_agent：基于创建日程 create_calendar_event 和查询空闲时间 get_available_time_slots 两个工具实现精准的日常管理工作。
- 邮件发送子智能体 email_agent：基于发送邮件 send_email 工具实现对于指定用户发送特定内容的信息的工作。

和之前构建智能体一样，我们需要先去设置一下其所使用的大模型、工具以及系统提示词（子智能体由于只需要完成指定任务不需要设置记忆）。

#### 构建日程管理子智能体 calendar_agent：

In [13]:
from langchain_community.chat_models import ChatTongyi
model = ChatTongyi(api_key=os.environ.get("DASHSCOPE_API_KEY"), model="qwen-max")

calendar_agent = create_agent(
    model,
    tools=[create_calendar_event, get_available_time_slots],
    system_prompt=(
    "You are a calendar scheduling assistant. "
    "Parse natural language scheduling requests (e.g., 'next Tuesday at 2pm') "
    "into proper ISO datetime formats. "
    "Use get_available_time_slots to check availability when needed. "
    "Use create_calendar_event to schedule events. "
    "Always confirm what was scheduled in your final response."
    )
)

然后我们可以测试一下其具体的效果（这里使用的是流式输出的方式）：

In [14]:
query = "Schedule a team meeting next Tuesday at 2pm for 1 hour"

for step in calendar_agent.stream({"messages": [{"role": "user", "content": query}]}):
    for update in step.values():
        for message in update.get("messages", []):
            message.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  create_calendar_event (call_63ed3cb311ca4b20b873ff)
 Call ID: call_63ed3cb311ca4b20b873ff
  Args:
    title: Team Meeting
    start_time: 2023-11-21T14:00:00
    end_time: 2023-11-21T15:00:00
    attendees: ['team@company.com']
    location:
================================= Tool Message =================================
Name: create_calendar_event

Event created: Team Meeting from 2023-11-21T14:00:00 to 2023-11-21T15:00:00 with 1 attendees
================================== Ai Message ==================================

The team meeting has been scheduled for next Tuesday, November 21, 2023, from 2:00 PM to 3:00 PM.


#### 构建发送邮件子智能体 email_agent：


In [15]:
from langchain_community.chat_models import ChatTongyi
model = ChatTongyi(api_key=os.environ.get("DASHSCOPE_API_KEY"), model="qwen-max")

email_agent = create_agent(
    model,
    tools=[send_email],
    system_prompt=(
    "You are an email assistant. "
    "Compose professional emails based on natural language requests. "
    "Extract recipient information and craft appropriate subject lines and body text. "
    "Use send_email to send the message. "
    "Always confirm what was sent in your final response."
    )
)

然后我们也可以尝试着对其进行调用：

In [16]:
query = "Send the design team a reminder about reviewing the new mockups"

for step in email_agent.stream({"messages": [{"role": "user", "content": query}]}):
    for update in step.values():
        for message in update.get("messages", []):
            message.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  send_email (call_a1ee65f3a51942d4b56750)
 Call ID: call_a1ee65f3a51942d4b56750
  Args:
    to: ['design.team@company.com']
    subject: Reminder: Review New Mockups
    body: Hello Design Team, Just a friendly reminder to review the new mockups at your earliest convenience. Your feedback is greatly appreciated. Best, [Your Name]
================================= Tool Message =================================
Name: send_email

Email sent to design.team@company.com - Subject: Reminder: Review New Mockups
================================== Ai Message ==================================

The email has been sent to the design team with the subject "Reminder: Review New Mockups". The body of the email kindly asks them to review the new mockups at their earliest convenience and appreciates their feedback.


### 2.3.3 将子代理包装成工具（给 Supervisor 调用）
虽然 calendar_agent 和 email_agent 已经可以独立工作，但 Supervisor_agent 是通过「调用工具」的方式来使用它们的。所以我们还要把子代理包装成新的高层工具，供上层 Supervisor_agent 调度。

所以我们刚刚的两个 agent 会被包装成两个工具：
- schedule_event
- manage_email

当然我们也需要写上对应的函数文档字符串来告诉 Supervisor 在什么时候去使用该工具。

#### 构建子代理工具 schedule_event：

In [17]:
@tool
def schedule_event(request: str) -> str:
  """Schedule calendar events using natural language.

  Use this when the user wants to create, modify, or check calendar appointments.
  Handles date/time parsing, availability checking, and event creation.

  Input: Natural language scheduling request (e.g., 'meeting with design team
  next Tuesday at 2pm')
  """
  result = calendar_agent.invoke({
    "messages": [{"role": "user", "content": request}]
  })
  return result["messages"][-1].text

#### 构建子代理工具 manage_email：

In [18]:
@tool
def manage_email(request: str) -> str:
  """Send emails using natural language.

  Use this when the user wants to send notifications, reminders, or any email
  communication. Handles recipient extraction, subject generation, and email
  composition.

  Input: Natural language email request (e.g., 'send them a reminder about
  the meeting')
  """
  result = email_agent.invoke({
    "messages": [{"role": "user", "content": request}]
  })
  return result["messages"][-1].text

### 2.3.4 构建 Supervisor Agent（调度者）

在构建好了 Supervisor 可使用的工具后，我们就可以创建 Supervisor_agent 并准备后续进行调用了，其具体的职责并不是直接使用底层工具，而是：
- 理解用户自然语言请求
- 拆解任务
- 调用合适的子代理工具
- 整合并回复结果

所以根据这个需求，我们可以按以下方式创建 Supervisor_agent :

In [19]:
from langchain_community.chat_models import ChatTongyi
model = ChatTongyi(api_key=os.environ.get("DASHSCOPE_API_KEY"), model="qwen-max")

supervisor_agent = create_agent(
  model,
  tools=[schedule_event, manage_email],
  system_prompt=(
    "You are a helpful personal assistant. "
    "You can schedule calendar events and send emails. "
    "Break down user requests into appropriate tool calls and coordinate the results. "
    "When a request involves multiple actions, use multiple tools in sequence."
  )
)

### 2.3.5 调用 Supervisor Agent

假如我们就希望其进行会议的安排：

In [20]:
query = "Schedule a team standup for tomorrow at 9am"

for step in supervisor_agent.stream(
    {"messages": [{"role": "user", "content": query}]}
):
    for update in step.values():
        for message in update.get("messages", []):
            message.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  schedule_event (call_c97703b9ea474744ba2a06)
 Call ID: call_c97703b9ea474744ba2a06
  Args:
    request: Schedule a team standup for tomorrow at 9am
================================= Tool Message =================================
Name: schedule_event

The team standup has been scheduled for tomorrow at 9:00 AM and will last until 9:30 AM. All team members have been invited.
================================== Ai Message ==================================

The team standup has been scheduled for tomorrow at 9:00 AM and will last until 9:30 AM. All team members have been invited.


那这个时候就会单独只调用其中一个工具（schedule_event）来进行完成。

假如我们希望其能够其能够同时安排会议并发邮件提醒，我们也可以进行实现：

In [21]:
query = (
    "Schedule a meeting with the design team next Tuesday at 2pm for 1 hour, "
    "and send them an email reminder about reviewing the new mockups."
)

for step in supervisor_agent.stream(
    {"messages": [{"role": "user", "content": query}]}
):
    for update in step.values():
        for message in update.get("messages", []):
            message.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  schedule_event (call_f30615fe388242c48aac91)
 Call ID: call_f30615fe388242c48aac91
  Args:
    request: Schedule a meeting with the design team next Tuesday at 2pm for 1 hour
  manage_email (call_974f3de7b3ba46ab81664c)
 Call ID: call_974f3de7b3ba46ab81664c
  Args:
    request: send an email reminder to the design team about reviewing the new mockups
================================= Tool Message =================================
Name: manage_email

The email reminder has been sent to the design team at design.team@example.com. The subject of the email is "Reminder: Review New Mockups" and it kindly asks them to review the new mockups and share their feedback by the end of the day tomorrow.
================================= Tool Message =================================
Name: schedule_event

The meeting with the design team has been scheduled for next Tuesday, November 21, from 2:00 PM to 3:0

可以看到返回的内容为：
- Supervisor_agent 分析了任务后一次性发出了两条工具调用的指令
- 然后子代理 schedule_event 和 manage_email 分别返回了工具调用的结果（非直接工具返回的结果）
- 最后 Supervisor_agent 根据工具调用的信息对问题进行了回复

## 2.4 总结

整个 Tool Calling 模式的 Multi-Agent System 由三层组成：
- 底层：是一些严格格式的 API 工具，它们要求精确的输入格式；
- 中间层：是子代理（Sub-agent），可以接收自然语言请求，将其转换为结构化 API 调用，并返回自然语言的确认回复；
- 顶层：是 Supervisor（监督者代理），负责调用这些高层功能（子代理工具）并整合最终结果。

这种“职责分离”的架构带来了多个优势：
- 每一层都有清晰、专注的职责；
- 可以在不影响已有模块的前提下，轻松添加新领域；
- 各层之间解耦，便于独立测试与持续优化。

## 2.5 多智能体优化

假如我们希望对当前 Tool Calling 模式的 Multi-Agent System 进行优化，我们可以从三个方面入手：
- 增加子智能体能够获取到的信息
- 增加主智能体能够从子智能体中获取到的信息（也就是子智能体返回更多信息）
- 对于一些危险的工具添加人工审核以避免风险
|
### 2.5.1 增加子智能体获取的信息

在当前的代码里， sub_agent 能看到的信息只是 Supervisor_agent 发送的查询请求。但假如我们希望让子代理看到“上层全部对话历史”或其它状态信息，我们可以通过工具调用时的 ToolRuntime 注入状态：

In [22]:
from langchain.tools import tool, ToolRuntime

@tool
def schedule_event(request: str, runtime: ToolRuntime) -> str:
  """Schedule calendar events using natural language.

  Use this when the user wants to create, modify, or check calendar appointments.
  Handles date/time parsing, availability checking, and event creation.

  Input: Natural language scheduling request (e.g., 'meeting with design team
  next Tuesday at 2pm')
  """
  # Customize context received by sub-agent
  original_user_message = next(message for message in runtime.state["messages"] if message.type == "human")
  prompt = ("You are assisting with the following user inquiry:\n\n"
      f"{original_user_message.text}\n\n"
      "You are tasked with the following sub-request:\n\n"
      f"{request}")
  result = calendar_agent.invoke({"messages": [{"role": "user", "content": prompt}]})
  return result["messages"][-1].text

所以这个时候 sub_agent 不再只能接收到调用（request）的信息，还能知道部分当前任务的信息并智能化的做出调用的决定。

### 2.5.2 增加主智能体获取的信息

除了向子代理里注入更多的信息，类似的我们可以让子代理返回更多的信息，比如在 return 的时候并不是仅仅把 result["messages"][-1].text 而是返回一些状态信息：

In [23]:
@tool
def schedule_event(request: str) -> str:
    """Schedule calendar events using natural language.

    Use this when the user wants to create, modify, or check calendar appointments.
    Handles date/time parsing, availability checking, and event creation.

    Input: Natural language scheduling request (e.g., 'meeting with design team
    next Tuesday at 2pm')
    """
    result = calendar_agent.invoke({"messages": [{"role": "user", "content": request}]})

    # Option 1: Return just the confirmation message
    return result["messages"][-1].text

    # Option 2: Return structured data
    # return json.dumps({
    #     "status": "success",
    #     "event_id": "evt_123",
    #     "summary": result["messages"][-1].text
    # })

通过 json.dumps ，我们可以把特定格式的 json 内容转为字符串的形式然后再返回给大模型。比如上面的内容会得到： ``` "{\"status\": \"success\", \"event_id\": \"evt_123\", \"summary\": \"The event has been scheduled for next Tuesday at 2 PM.\"}" ```。这样Supervisor_agent 就会知道成功运行且运行的 id 是 evt_123 了。

### 2.5.3 添加人工审核
对于人工审核，可以使用上一章节提到的内置中间件 HumanInTheLoopMiddleware 来实现。这个中间件允许我们能够在“子代理调用工具之前”插入人工判断点。

比如当智能体执行关键操作（如发邮件、创建日程）时，先中断流程，由人类手动“审批 / 编辑 / 拒绝”，再继续执行。

那在我们的系统中，我们也可以添加进该机制，从而实现：
- 确保发出去的邮件不会出错
- 日程安排前想确认时间和人选
- 涉及敏感操作（下单、删库、通知大群）能够提前预判

我们可以给 calendar_agent 添加上人工审核中间件。仅针对 create_calendar_event 工具，而 get_available_time_slots 工具可以自由调用。

In [29]:
from langchain.agents.middleware import HumanInTheLoopMiddleware

calendar_agent = create_agent(
  model,
  tools=[create_calendar_event, get_available_time_slots],
  system_prompt=("You are a calendar scheduling assistant..."),
  middleware=[ 
    HumanInTheLoopMiddleware( 
      interrupt_on={"create_calendar_event": True}, 
      description_prefix="Calendar event pending approval", 
    ), 
  ], 
)

而对于第二个 email_agent 而言，也可以对 send_email 添加上人工审核。

In [30]:
email_agent = create_agent(
  model=llm,
  tools=[send_email],
  system_prompt=("You are an email assistant... "),
  middleware=[ 
    HumanInTheLoopMiddleware( 
      interrupt_on={"send_email": True}, 
      description_prefix="Outbound email pending approval", 
    ), 
  ], 
)

但是在 HumanInTheLoopMiddleware 中有一个要求，就是必须要在有设置记忆的情况下才能够实现人工审核的操作。所以我们可以为 supervisor_agent 添加 checkpointer 参数：

In [31]:
from langgraph.checkpoint.memory import InMemorySaver

supervisor_agent = create_agent(
    model=model,
    tools=[schedule_event, manage_email],
    system_prompt=("You are a helpful personal assistant. "),
    checkpointer=InMemorySaver()  # 🔥 激活状态保存
)

这样子，在运行过程中的信息就能够保存下来了，那么在传入人工的指令后也可以基于之前的记忆继续恢复了。

对于子代理而言，由于其本身就是为了完成任务而设置的，除非是一些需要基于上下文完成任务的智能体工具，不然是不需要添加记忆进去的。添加了反而可能导致状态不一致或冗余。

当然和前面一样，我们一样可以提出问题，然后调用 supervisor_agent 进行回复：

In [32]:
from langchain_core.messages import HumanMessage

query = (
    "Schedule a meeting with the design team next Tuesday at 2pm for 1 hour, "
    "and send them an email reminder about reviewing the new mockups."
)

config = {"configurable": {"thread_id": "1"}}

response = supervisor_agent.invoke(
    {"messages": [HumanMessage(content=query)]},
    config=config
)

print(response)

{'messages': [HumanMessage(content='Schedule a meeting with the design team next Tuesday at 2pm for 1 hour, and send them an email reminder about reviewing the new mockups.', additional_kwargs={}, response_metadata={}, id='ef8d6bee-a439-4136-9f0a-68f6e24af783'), AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"request": "Schedule a meeting with the design team next Tuesday at 2pm for 1 hour"}', 'name': 'schedule_event'}, 'id': 'call_2486f5f6c2184436a6d0ed', 'index': 0, 'type': 'function'}, {'function': {'arguments': '{"request": "send an email reminder to the design team about reviewing the new mockups before the meeting"}', 'name': 'manage_email'}, 'id': 'call_f13b3c94f87f4867869645', 'index': 1, 'type': 'function'}]}, response_metadata={'model_name': 'qwen-max', 'finish_reason': 'tool_calls', 'request_id': '8de8be86-8eab-4955-9a3b-54d404fc8abd', 'token_usage': {'input_tokens': 431, 'output_tokens': 66, 'prompt_tokens_details': {'cached_tokens': 0}

In [33]:
from langgraph.types import Command
# 🔁 循环处理多个中断（逐个审批直到流程结束）
while "__interrupt__" in response:
  print("\n🟠 检测到中断，进入人工审核环节...")

  # 🧑 打印用户原始提问
  user_msg = next((m.content for m in response["messages"] if m.type == "human"), "无")
  print(f"\n🧑 用户提问：{user_msg}")

  # 遍历中断对象（可能包含多个 action request）
  for interrupt in response["__interrupt__"]:
    for i, req in enumerate(interrupt.value["action_requests"]):
      print(f"📝 描述：{req.get('description', '无')}")
      print(f"✅ 可选操作：{interrupt.value['review_configs'][i]['allowed_decisions']}")

    # 👤 人工输入决策
    decision_type = input("\n请输入你的决定（approve / edit / reject）：").strip().lower()
    if decision_type not in ["approve", "edit", "reject"]:
      print("⚠️ 无效输入，默认设置为 reject")
      decision_type = "reject"

    decision_payload = {"type": decision_type}

    # ✍️ 如果选择 edit：展示原请求并让用户直接输入完整 JSON
    if decision_type == "edit":
      print("\n🛠️ 当前模型原始调用指令如下：")
      print(json.dumps(req, indent=2, ensure_ascii=False))
      print("\n✍️ 请粘贴你想执行的完整新调用 JSON（例如：）")
      print('{"name": "send_email", "args": {"to": ["x@example.com"], "subject": "测试", "body": "内容"}}')
      try:
        new_raw = input("\n请输入新的调用 JSON：").strip()
        decision_payload["edited_action"] = json.loads(new_raw)
      except Exception as e:
        print(f"❌ JSON 格式错误：{e}")
        print("⚠️ 自动拒绝此调用")
        decision_payload = {"type": "reject", "message": "编辑失败，输入格式错误"}
    # ❌ 如果拒绝，填写理由
    elif decision_type == "reject":
      reason = input("请输入拒绝理由：")
      decision_payload["message"] = reason
    # ✅ 恢复执行（Resume）
    try:
      response = supervisor_agent.invoke(
        Command(resume={interrupt.id: {"decisions": [decision_payload]}}), config=config)
    except Exception as e:
      print(f"❌ 执行恢复时发生错误：{e}")
      break
# ✅ 最终结果输出
print("\n🟢 流程结束，模型最终回复：")
print(response["messages"][-1].content)


🟠 检测到中断，进入人工审核环节...

🧑 用户提问：Schedule a meeting with the design team next Tuesday at 2pm for 1 hour, and send them an email reminder about reviewing the new mockups.
📝 描述：Outbound email pending approval

Tool: send_email
Args: {'body': 'Please review the new mockups before our meeting.', 'cc': [], 'subject': 'Email Reminder: Review New Mockups Before Meeting', 'to': ['design-team@example.com']}
✅ 可选操作：['approve', 'edit', 'reject']
📝 描述：Calendar event pending approval

Tool: create_calendar_event
Args: {'title': 'Design Team Meeting', 'start_time': '2023-10-10T14:00:00', 'end_time': '2023-10-10T15:00:00', 'attendees': ['designer1@example.com', 'designer2@example.com'], 'location': 'Conference Room B'}
✅ 可选操作：['approve', 'edit', 'reject']

🟢 流程结束，模型最终回复：
The meeting with the design team has been scheduled for next Tuesday, October 10th, from 2 PM to 3 PM in Conference Room B. All team members have been notified, and an email reminder about reviewing the new mockups before the meeting has